In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import time
from selenium import webdriver
import zipfile
import pandas as pd

Buenos días, el cliente nos ha dado nuevos detalles sobre la prueba, me gustaría que me confirmaras que tu prueba está completa con estos detalles, si es así perfecto, si no, puedes realizar lo que te falte y me envías nuevamente para revisar, a continuación la información:

- Deben ser desarrollados los siguientes puntos completamente en python

1. Ir a:

https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/home

2. Sección "Documents & Downloads"

3. En la sección "National Incident-Based Reporting System (NIBRS) Tables" elegir table: Victims, año: 2022, location: Florida

4. Descargar archivo

5. Con pandas leer zip y elegir el archivo: Victims_Age_by_Offense_Category_2022.xlsx

6. Elegir categoria Crimes Against Property y generar csv sin totales, footer, ni index

- Se requiere un código que ejecute todas las partes, desde el webscraping hasta la generación del archivo csv, debe entregar el código completo y el archivo resultado.



Quedo atento, muchas gracias por tu tiempo

In [40]:
# Configura el driver de Selenium (asegúrate de tener el driver correspondiente descargado)
driver = webdriver.Chrome()  # o webdriver.Firefox() si prefieres Firefox

# Abre la página web
url = 'https://cde.ucr.cjis.gov/LATEST/webapp/#/pages/home'
driver.get(url)

# Espera hasta que el botón esté presente y visible antes de hacer clic en él
try:
    # Espera un máximo de 10 segundos para encontrar el botón y luego haz clic en él
    WebDriverWait(driver, 6).until(
        EC.element_to_be_clickable((By.ID, 'home-dwnload-go-btn'))
    ).click()
    print("Se hizo clic en el botón 'Go'")


    dropdown = WebDriverWait(driver, 4).until(
        EC.element_to_be_clickable((By.ID, 'dwnnibrs-download-select'))
    )
    dropdown.click()
    # Seleccionamos la categoria Victims
    option_victims = WebDriverWait(driver, 4).until(
        EC.element_to_be_clickable((By.XPATH, "//nb-option[contains(text(), 'Victims')]"))
    )
    option_victims.click()


    dropdown_2 = WebDriverWait(driver, 4).until(
        EC.element_to_be_clickable((By.XPATH, "//nb-select[@id='dwnnibrsloc-select']"))
    )
    dropdown_2.click()

    # Encuentra y haz clic en la opción "Florida" en el segundo menú desplegable
    option_florida = WebDriverWait(driver, 2).until(
        EC.element_to_be_clickable((By.XPATH, "//nb-option[contains(text(), 'Florida')]"))
    )
    option_florida.click()

    WebDriverWait(driver, 2).until(
        EC.element_to_be_clickable((By.ID, 'nibrs-download-button'))
    ).click()

    time.sleep(5)


except Exception as e:
    print("No se pudo realizar la selección:", e)

# Cierra el navegador cuando hayas terminado
driver.quit()


Se hizo clic en el botón 'Go'


In [41]:

# Función para obtener las carpetas de descargas según el sistema operativo
def obtener_carpetas_descargas():
    sistema_operativo = os.name
    if sistema_operativo == 'posix':  # Sistema tipo Unix (Linux, macOS, etc.)
        return [os.path.expanduser('~/Downloads')]
    elif sistema_operativo == 'nt':  # Windows
        return [os.path.join(os.environ['USERPROFILE'], 'Downloads')]
    else:
        return []  # No se pudo determinar el sistema operativo

# Función para buscar el archivo en las carpetas de descargas
def buscar_archivo_en_carpetas(nombre_archivo, carpetas):
    archivos_encontrados = []
    for carpeta in carpetas:
        for ruta_carpeta, _, archivos in os.walk(carpeta):
            if nombre_archivo in archivos:
                archivos_encontrados.append(os.path.join(ruta_carpeta, nombre_archivo))
    return archivos_encontrados

# Nombre del archivo que estás buscando
nombre_archivo = 'victims.zip'

# Obtener las carpetas de descargas según el sistema operativo
carpetas_descargas = obtener_carpetas_descargas()

# Buscar el archivo en las carpetas de descargas
archivos_encontrados = buscar_archivo_en_carpetas(nombre_archivo, carpetas_descargas)

archivos_encontrados

['C:\\Users\\jonat\\Downloads\\victims.zip']

In [42]:

# Abre el archivo ZIP en modo lectura
with zipfile.ZipFile(archivos_encontrados[0], 'r') as archivo_zip:
    
    lista_contenidos = archivo_zip.namelist()
    # Extrae el excel solicitado
    archivo_zip.extract('Victims_Age_by_Offense_Category_2022.xlsx')

In [47]:
victims = pd.read_excel('Victims_Age_by_Offense_Category_2022.xlsx')
victims.drop(victims.index[:2], inplace=True)
victims.drop(victims.index[-1], inplace=True)
indice_age = victims.iloc[1].values[2:]
indice_age = [x for x in indice_age]
indice_age.insert(0,'Total_Victims')
indice_age.insert(0,'Offense Category')
indice_age = [columna.replace('\n', ' ') if isinstance(columna, str) else columna for columna in indice_age]
Crimes_Against_Property = victims.loc[victims['Victims'] == 'Crimes Against Property'].copy()
Crimes_Against_Property.columns = indice_age[0:len(Crimes_Against_Property.columns)]
Crimes_Against_Property.drop(columns=['Total_Victims'], inplace=True)
Crimes_Against_Property = Crimes_Against_Property.transpose()
Crimes_Against_Property.drop(Crimes_Against_Property.index[0], inplace=True)
Crimes_Against_Property.columns = ['Crimes Against Property']
Crimes_Against_Property = Crimes_Against_Property.rename_axis('Age')
Crimes_Against_Property.to_csv('Crimes_Against_Property.csv')